# Light Curve Reports Outline

For each potential reference star, report the following:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
# Import lightkurve
import lightkurve as lk
import astropy.units as u
import astropy.constants as const
# Import astropy's tables
from astropy.table import QTable
# Import yaml
import yaml
# Import os 
import os
from pprint import pprint

In [2]:
## Setting up the directories for saving
#
# Git directory name
gitdir = "roman-vetting"
# Get the present working directory
pwdir = os.getcwd()
# Determine the path to the base of the git repository
basedir = pwdir.split(gitdir)[0] + gitdir + "/"
# Set the path to the data directory
datadir = basedir + "data/"
# Set the path to the candidate reports directory
canddir = datadir + "candidate_reports/"
# Create this directory if it doesn't exist
if not os.path.isdir(canddir):
    os.mkdir(canddir)
# Set this as the save directory
savedir = canddir

print("Saving to {}".format(savedir))

Saving to /home/ntschragal/uasal/roman-vetting/data/candidate_reports/


# Prepare the list of current candidates

In [3]:
# Load the current candidate list
with open(canddir + "current_list.yaml", 'r') as f:
    candlist = yaml.load(f, Loader=yaml.FullLoader)

In [4]:
for c in candlist:
    # Create a separate directory for each candidate
    candidate_dir = (savedir + c + "/")
    # If the directory doesn't exist, create it
    if not os.path.isdir(candidate_dir):
        os.mkdir(candidate_dir)

## Lightkurve data retrieval and display

Well, that's a lot of data. A lot of it should go in the image metadata. Actually, all of it can go in the image metadata. So here's what the image information will look like:

- Sub-directory: lcs
- Name: [project]\_[author]\_[year].png

In [5]:
for c in candlist:
    # Search the lightkurve catalogs
    print("Searching for light curve data for {}... ".format(c.upper()), end = "")
    sr = lk.search_lightcurve(c)
    # Convert the search results to a table
    srtable = sr.table
    print("Found {} results".format(len(srtable)))
    if len(srtable) == 0:
        print("Did not find any light curve results for {}, continuing to next candidate...".format(c))
        print("============================================", end = "\n\n")
        continue

    # If not present, create a direcotry for the light curve data to be downloaded into
    canddir = (savedir + c + "/")
    lc_data_dir = (canddir + "lc_data/")
    if not os.path.isdir(lc_data_dir):
        os.mkdir(lc_data_dir)

    # If not present, create a directory for the light curve figures to be saved into
    lc_fig_dir = (canddir + "lc_figs/")
    if not os.path.isdir(lc_fig_dir):
        os.mkdir(lc_fig_dir)

    # Additionally, create a directory for the light curve figure metadata to be saved into
    lc_meta_dir = (lc_fig_dir + "lc_meta/")
    if not os.path.isdir(lc_meta_dir):
        os.mkdir(lc_meta_dir)
    
    print("Light curves will be downloaded into {} and figures will be saved into {}.".format(lc_data_dir, lc_fig_dir))

    # For each result in the search results, download the light curve, save the data, and plot the output
    for i in range(len(srtable)):

        # Get the row of data from srtable, convert it to a dictionary, and extract some metadata for file names
        try:
            row = srtable[i]
            metadict = dict(zip(row.colnames, row.values()))
            project = str(metadict['project'])
            author = str(metadict['author'])
            year = str(metadict['year'])
            obs_start = str(int(metadict['t_min']))
        except Exception as e:
            print("Problem encountered for {} result {}: {}".format(c, i, str(e)))
            continue

        # First, verify that the dataproduct is a timeseries. If not, put an error in the metadata, save it, and move on.
        if not srtable['dataproduct_type'][i] == 'timeseries':
            metadict['ERROR'] = "Skipping this result because it is not a timeseries.".format(c, i)
            savemetadict = {k: str(v) for k, v in metadict.items()}
            yamlpath = (lc_meta_dir + project + "_" + author + "_" + year + "_" + obs_start +".yaml")
            with open(yamlpath, 'w') as outfile:
                yaml.dump(savemetadict, outfile, default_flow_style = False)
            print(metadict['ERROR'])
            #print("============================================", end = "\n\n")
            continue
        
        try:
            # Download the light curve
            print("Downloading data for search result {}... ".format(i), end = "")
            lc_raw = sr[i].download(download_dir = lc_data_dir)
            print("Download complete.")
            # Get the normalization factor for the LC
            norm_factor = np.median(lc_raw.flux)
            lc = lc_raw.normalize()
            # Get the lomb-scargle periodogram
            pgram = lc.to_periodogram(
                method = 'lombscargle',
            #    normalization = 'psd',
            )
            # Get the period at maximum power
            pmaxpower = pgram.period_at_max_power
            metadict['norm_factor'] = norm_factor
            metadict['pmaxpower'] = pmaxpower
            # Get the 'project', 'author', and 'year' fields from the table as strings

            # Remove that principal signal from the light curve (roughly)
            lc_model = pgram.model(time = lc.time, frequency = pgram.frequency_at_max_power)
            newlc = lc.copy()
            newlc.flux = newlc.flux / lc_model.flux
            
            # Establish the plot filename
            plotpath = (lc_fig_dir + project + "_" + author + "_" + year + "_" + obs_start + ".png")

        except Exception as e:
            metadict['ERROR'] = "An error occurred in downloading the data for {} result {}: {}".format(c, i, str(e))
            savemetadict = {k: str(v) for k, v in metadict.items()}
            yamlpath = (lc_meta_dir + project + "_" + author + "_" + year + "_" + obs_start +".yaml")
            with open(yamlpath, 'w') as outfile:
                yaml.dump(savemetadict, outfile, default_flow_style = False)
            print(metadict['ERROR'])
            #print("============================================", end = "\n\n")
            continue


        try:
            nrows = 3
            ncols = 3

            fig = plt.figure(
                figsize = (16, 12),
                dpi = 200,
                tight_layout = True,
            )

            gs = GridSpec(
                nrows,
                ncols,
                figure = fig,
                width_ratios = [2, 1, 1],
            )

            # Plot the light curve
            ax = fig.add_subplot(gs[0, :])
            lc.scatter(
                ax = ax,
                marker = '.',
                s = 0.5,
                color = 'black',
                label = "Normalization = " + f"{norm_factor.value:0.2e} [{norm_factor.unit:latex}]",
            )
            ax.set_title(c.upper() + " " + project + " " + author + " (" + year + ") - Light Curve")
            ax.legend(
                loc = 'best',
                fontsize = 8,
            )
            #ax.set_ylabel("Normalized Flux (norm. = " + f"{norm_factor.value:0.2e} [{norm_factor.unit:latex}]" + ")")

            # Plot a periodogram of the above light curve
            ax = fig.add_subplot(gs[1, 0])
            pgram.plot(
                ax = ax,
                color = 'black',
                label = None,
            )
            ax.axvline(
                1/pmaxpower.to(u.day).value,
                color = 'red',
                alpha = 1,
                linestyle = '-',
                linewidth = 1,
                ymin = 0.0,
                ymax = 0.2,
                label = "p (max power): {:.2f} days".format(pmaxpower.to(u.day).value),
                zorder = 10,
            )
            ax.axvline(
                1/pmaxpower.to(u.day).value,
                color = 'red',
                alpha = 0.5,
                linestyle = ':',
                linewidth = 1,
                ymin = 0.2,
                ymax = 1.0,
                #label = "P (max power): {:.2f} days".format(pmaxpower.to(u.day).value),
                zorder = 10,
            )
            ax.set_yscale('log')
            ax.set_title("Periodogram (Lomb-Scargle)")
            ax.legend()

            # Plot a phase-folded light curve based on the period of maximum power
            ax = fig.add_subplot(gs[1, 1])
            lc.fold(
                period = pmaxpower.to(u.day).value,
            ).scatter(
                ax = ax,
                marker = '.',
                s = 0.5,
                color = 'black',
                label = None,
            )
            ax.set_title("Folded LC (p = {:0.2f} d)".format(pmaxpower.to(u.day).value))

            # Plot a river plot based on the period of maximum power
            ax = fig.add_subplot(gs[1, 2])
            lc.plot_river(
                ax = ax,
                period = pmaxpower.to(u.day).value,
            #    aspect = 'auto',
            )
            ax.set_aspect('auto')
            ax.set_title("River Plot (p = {:0.2f} d)".format(pmaxpower.to(u.day).value))

            # Plot the light curve with the principal signal removed
            ax = fig.add_subplot(gs[2, :])
            newlc.scatter(
                ax = ax,
                marker = '.',
                s = 0.5,
                color = 'black',
                label = None,
            )
            ax.set_title("Light Curve - Highest power frequency removed")
            newlc.bin(pmaxpower.value).plot(
                ax = ax,
                color = 'red',
                linewidth = 2.5,
                label = "Binned (size = {:.2f} d)".format(pmaxpower.to(u.day).value),
            )
            ax.legend(
                loc = 'best',
                fontsize = 8,
            )


            # Turn each piece of the metadata into strings for saving with a dictionary comprehension
            savemetadict = {k: str(v) for k, v in metadict.items()}

            # Save the figure
            fig.savefig(
                plotpath,
                metadata = savemetadict,
            )

            # Save the metadata as a yaml file
            yamlpath = (lc_meta_dir + project + "_" + author + "_" + year + "_" + obs_start +".yaml")
            with open(yamlpath, 'w') as outfile:
                yaml.dump(savemetadict, outfile, default_flow_style = False)

            #plt.show()
            plt.close()
        
        except Exception as e:
            metadict['ERROR'] = "An error occurred in creating the plot for {} result {}: {}".format(c, i, str(e))
            savemetadict = {k: str(v) for k, v in metadict.items()}
            yamlpath = (lc_meta_dir + project + "_" + author + "_" + year + "_" + obs_start +".yaml")
            with open(yamlpath, 'w') as outfile:
                yaml.dump(savemetadict, outfile, default_flow_style = False)
            print(metadict['ERROR'])
            print("============================================", end = "\n\n")
            plt.close()
            continue
            
        
    
    print("============================================", end = "\n\n")


Searching for light curve data for HD432... Found 15 results
Light curves will be downloaded into /home/ntschragal/uasal/roman-vetting/data/candidate_reports/hd432/lc_data/ and figures will be saved into /home/ntschragal/uasal/roman-vetting/data/candidate_reports/hd432/lc_figs/.


/home/ntschragal/anaconda3/envs/psfvet/lib/python3.12/site-packages/astropy/units/quantity.py:666: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [34]:
sr = lk.search_lightcurve(candlist[0])

srtable = sr.table

In [35]:
srtable

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID,objID1,distance,exptime,obsID,obs_collection_products,dataproduct_type_products,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project_products,prvversion,proposal_id_products,productFilename,size,parent_obsid,dataRights_products,calib_level_products,filters_products,author,mission,#,year,sort_order
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcsec,s,,,,,,,,,,,,,,,,,,,,,,,,
str7,str4,str9,str10,str4,str4,str7,str9,str1,str58,float64,float64,str10,str19,int64,float64,float64,float64,float64,float64,str1,float64,str7,str1,int64,str47,str1,str116,str6,bool,float64,str9,str9,str9,float64,float64,str9,str4,str10,str33,str1,str116,str7,str28,str3,str1,str9,str20,str7,str63,int64,str9,str6,int64,str4,str9,object,int64,int64,int64
science,TESS,SPOC,Photometer,TESS,TESS,Optical,396298498,--,tess2019279210107-s0017-0000000396298498-0161-s,2.294576,59.149788,timeseries,"Ricker, George",3,58764.18633425,58789.18096395,120.0,600.0,1000.0,--,58818.3333334,G022133,--,17,CIRCLE ICRS 2.29457600 59.14978800 0.00138889,--,mast:TESS/product/tess2019279210107-s0017-0000000396298498-0161-s_lc.fits,PUBLIC,False,nan,27557684,70698409,70698409,0.0,120,27557684,TESS,timeseries,Light curves,S,mast:TESS/product/tess2019279210107-s0017-0000000396298498-0161-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.28-20200407,G022133,tess2019279210107-s0017-0000000396298498-0161-s_lc.fits,1846080,27557684,PUBLIC,3,TESS,SPOC,TESS Sector 17,0,2019,1
science,TESS,SPOC,Photometer,TESS,TESS,Optical,396298498,--,tess2019306063752-s0018-0000000396298498-0162-s,2.294576,59.149788,timeseries,"Ricker, George",3,58790.15874083,58814.51658604,120.0,600.0,1000.0,--,58833.3333334,G022133,--,18,CIRCLE ICRS 2.29457600 59.14978800 0.00138889,--,mast:TESS/product/tess2019306063752-s0018-0000000396298498-0162-s_lc.fits,PUBLIC,False,nan,27584844,70752709,70752709,0.0,120,27584844,TESS,timeseries,Light curves,S,mast:TESS/product/tess2019306063752-s0018-0000000396298498-0162-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.29-20200410,G022133,tess2019306063752-s0018-0000000396298498-0162-s_lc.fits,1800000,27584844,PUBLIC,3,TESS,SPOC,TESS Sector 18,1,2019,1
science,TESS,SPOC,Photometer,TESS,TESS,Optical,396298498,--,tess2020106103520-s0024-0000000396298498-0180-s,2.294576,59.149788,timeseries,"Ricker, George",3,58955.29038071,58981.77657075,120.0,600.0,1000.0,--,59001.0,G022133,--,24,CIRCLE ICRS 2.29457600 59.14978800 0.00138889,--,mast:TESS/product/tess2020106103520-s0024-0000000396298498-0180-s_lc.fits,PUBLIC,False,nan,27405827,70395931,70395931,0.0,120,27405827,TESS,timeseries,Light curves,S,mast:TESS/product/tess2020106103520-s0024-0000000396298498-0180-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.36-20200520,G022133,tess2020106103520-s0024-0000000396298498-0180-s_lc.fits,1952640,27405827,PUBLIC,3,TESS,SPOC,TESS Sector 24,2,2020,1
science,TESS,SPOC,Photometer,TESS,TESS,Optical,396298498,--,tess2022273165103-s0057-0000000396298498-0245-s,2.294576,59.149788,timeseries,"Ricker, George",3,59852.856472974534,59881.61795792824,120.0,600.0,1000.0,--,59904.0,N/A,--,57,CIRCLE 2.294576 59.149788 0.00138889,--,mast:TESS/product/tess2022273165103-s0057-0000000396298498-0245-s_lc.fits,PUBLIC,False,nan,101307712,189088321,189088321,0.0,120,101307712,TESS,timeseries,Light curves,S,mast:TESS/product/tess2022273165103-s0057-0000000396298498-0245-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-5.0.77-20221025,N/A,tess2022273165103-s0057-0000000396298498-0245-s_lc.fits,2108160,101307712,PUBLIC,3,TESS,SPOC,TESS Sector 57,3,2022,1
sci

In [39]:
authors = np.unique(srtable['author'].data)
authors

masked_array(data=['QLP', 'SPOC', 'TESS-SPOC'],
             mask=[False, False, False],
       fill_value='N/A',
            dtype='<U9')